<a href="https://colab.research.google.com/github/subhadrashaw/Web-Scraping/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Extraction

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel("/content/Input.xlsx")

In [4]:
df.head(5)

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


extract article text from each URL

In [ ]:
pip install newspaper3k

In [ ]:
from newspaper import Article
import nltk
nltk.download('punkt')

In [7]:
def get_text(url):
  url1 = url
  article = Article(url1, language = 'en')
  try:
    article.download()  
    article.parse()
    article.nlp()
  except:
    print('***FAILED TO DOWNLOAD***', article.url)  
  return article.title, article.text


In [ ]:
df['Title'] = np.NaN
df['Text'] = np.NaN

for i in range(0,len(df)):
  df['Title'][i], df['Text'][i]=get_text(df['URL'][i])
  



In [9]:
df.head(5)

,URL_ID,URL,Title,Text
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...


Text preprocessing and cleaning

In [28]:
from google.colab import files

uploaded = files.upload()

Saving negative-words.txt to negative-words (1).txt
Saving positive-words.txt to positive-words (1).txt


In [ ]:
import glob
import os

base_dir = '/content/'
file_list = glob.glob(os.path.join(base_dir, "*.txt"))

stopword = []

for file_path in file_list:
   with open(file_path, 'r',encoding='latin-1') as f:
        stopword.append(f.read().lower().split())
print(stopword)

In [12]:
import re

def transform(text):
  txt = re.sub('[^a-zA-Z0-9]',' ',text)
  txt = txt.lower()
  txt = txt.split()
  txt = [word for word in txt if not word in stopword]   # remove stopwords.
  #txt = ' '.join(txt)
  return txt

df['Transformed_text'] = df['Text'].apply(transform)

In [13]:
df.head(5)

,URL_ID,URL,Title,Text,Transformed_text
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...,"[introduction, if, anything, kills, over, 10, ..."
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...","[human, minds, a, fascination, in, itself, car..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...,"[introduction, ai, is, rapidly, evolving, in, ..."
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,"[anything, that, could, give, rise, to, smarte..."
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,"[machine, intelligence, is, the, last, inventi..."


# Data Analysis


Creating a dictionary of Positive words

In [ ]:
with open("positive-words.txt",'r') as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_words = " ".join ([w for w in poswords if not w in stopword])
pos_words=pos_words.split(" ")
print(pos_words)

Calculation of Positive Score

In [ ]:
df['Positive Score']=np.NaN
for i in range(0,len(df)):
  pos_count = " ".join ([w for w in df['Transformed_text'][i] if w in pos_words])
  pos_count=pos_count.split(" ")
  df['Positive Score'][i]= len(pos_count)

In [16]:
df.head(5)

,URL_ID,URL,Title,Text,Transformed_text,Positive Score
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...,"[introduction, if, anything, kills, over, 10, ...",81.0
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...","[human, minds, a, fascination, in, itself, car...",72.0
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...,"[introduction, ai, is, rapidly, evolving, in, ...",74.0
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,"[anything, that, could, give, rise, to, smarte...",86.0
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,"[machine, intelligence, is, the, last, inventi...",71.0


Creating a dictionary of Negative words

In [ ]:
with open("negative-words.txt",'r',encoding='latin-1') as neg:
    negwords = neg.read().split("\n")  
    negwords = negwords[5:]
neg_words = " ".join ([w for w in negwords if not w in stopword])
neg_words=neg_words.split(" ")
print(neg_words)

Calculation of Negative Score

In [ ]:
df['Negative Score']=np.NaN
for i in range(0,len(df)):
  neg_count = " ".join ([w for w in df['Transformed_text'][i] if w in neg_words])
  neg_count=neg_count.split(" ")
  df['Negative Score'][i]= len(neg_count)

In [19]:
df.head(5)

,URL_ID,URL,Title,Text,Transformed_text,Positive Score,Negative Score
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...,"[introduction, if, anything, kills, over, 10, ...",81.0,35.0
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...","[human, minds, a, fascination, in, itself, car...",72.0,37.0
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...,"[introduction, ai, is, rapidly, evolving, in, ...",74.0,36.0
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,"[anything, that, could, give, rise, to, smarte...",86.0,28.0
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,"[machine, intelligence, is, the, last, inventi...",71.0,29.0


Calculation of Polarity and Subjectivity Score

In [ ]:
df['Polarity Score'] = np.nan
df['Subjectivity Score']=np.nan

for i in range(len(df)):
    df['Polarity Score'][i] = (df['Positive Score'][i]-df['Negative Score'][i])/ ((df['Positive Score'][i] + df['Negative Score'][i]) + 0.000001)
    df['Subjectivity Score'][i] = (df['Positive Score'][i]+df['Negative Score'][i])/ (len(df['Transformed_text'][i]) + 0.000001)

In [21]:
df.head(5)

,URL_ID,URL,Title,Text,Transformed_text,Positive Score,Negative Score,Polarity Score,Subjectivity Score
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...,"[introduction, if, anything, kills, over, 10, ...",81.0,35.0,0.396552,0.063562
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...","[human, minds, a, fascination, in, itself, car...",72.0,37.0,0.321101,0.075433
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...,"[introduction, ai, is, rapidly, evolving, in, ...",74.0,36.0,0.345455,0.064140
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,"[anything, that, could, give, rise, to, smarte...",86.0,28.0,0.508772,0.068841
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,"[machine, intelligence, is, the, last, inventi...",71.0,29.0,0.420000,0.055928


Average Number of Words Per Sentence

In [ ]:
# word count in each text row.

df['word_counts'] = df['Transformed_text'].apply(lambda x: len(str(x).split()))   

df['average number of words per sentence'] = np.nan

for i in range(0,len(df)):
    
    df['average number of words per sentence'][i] = df['word_counts'][i]/len(nltk.sent_tokenize(df['Text'][i]))

In [23]:
df.head(5)

,URL_ID,URL,Title,Text,Transformed_text,Positive Score,Negative Score,Polarity Score,Subjectivity Score,word_counts,average number of words per sentence
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...,"[introduction, if, anything, kills, over, 10, ...",81.0,35.0,0.396552,0.063562,1825,23.701299
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...","[human, minds, a, fascination, in, itself, car...",72.0,37.0,0.321101,0.075433,1445,18.062500
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...,"[introduction, ai, is, rapidly, evolving, in, ...",74.0,36.0,0.345455,0.064140,1715,20.176471
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,"[anything, that, could, give, rise, to, smarte...",86.0,28.0,0.508772,0.068841,1656,17.431579
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,"[machine, intelligence, is, the, last, inventi...",71.0,29.0,0.420000,0.055928,1788,22.632911


Syllable Count and Complex Word Count

In [ ]:
def syllable_count(word):
    tscount = 0
    cwcount = 0
    wscount = 0
    u = ""
    vowels = "aeiouy"
    for w in word:
      if w!=' ' :
        u+=w;
      if w in vowels:
        wscount += 1
      if w == ' ' :
        if u.endswith("es"or "ed"):
          wscount -= 1 
        if wscount >= 2:
          cwcount += 1
        tscount += wscount
        wscount = 0  
        u = ""
    return tscount, cwcount


df['Syllable Count']=np.NaN
df['Complex Word Count']=np.NaN

for i in range(0,len(df)):
  df['Syllable Count'][i], df['Complex Word Count'][i] = syllable_count(' '.join(df['Transformed_text'][i]))


In [30]:
df.head(5)

,URL_ID,URL,Title,Text,Transformed_text,Positive Score,Negative Score,Polarity Score,Subjectivity Score,word_counts,average number of words per sentence,Syllable Count,Complex Word Count,Personal Pronouns
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...,"[introduction, if, anything, kills, over, 10, ...",81.0,35.0,0.396552,0.063562,1825,23.701299,3952.0,1109.0,1.0
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...","[human, minds, a, fascination, in, itself, car...",72.0,37.0,0.321101,0.075433,1445,18.062500,2680.0,710.0,7.0
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...,"[introduction, ai, is, rapidly, evolving, in, ...",74.0,36.0,0.345455,0.064140,1715,20.176471,3641.0,987.0,3.0
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,"[anything, that, could, give, rise, to, smarte...",86.0,28.0,0.508772,0.068841,1656,17.431579,3181.0,882.0,18.0
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,"[machine, intelligence, is, the, last, inventi...",71.0,29.0,0.420000,0.055928,1788,22.632911,3433.0,925.0,16.0


Personal Pronoun

In [ ]:
import re

def pronoun_count(word):
  pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
  pronouns = pronounRegex.findall(word)
  return len(pronouns)

df['Personal Pronouns'] = np.NaN

for i in range (0,len(df)):
  df['Personal Pronouns'][i] = pronoun_count(' '.join(df['Transformed_text'][i]))


In [32]:
df.head(5)

,URL_ID,URL,Title,Text,Transformed_text,Positive Score,Negative Score,Polarity Score,Subjectivity Score,word_counts,average number of words per sentence,Syllable Count,Complex Word Count,Personal Pronouns
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...,"[introduction, if, anything, kills, over, 10, ...",81.0,35.0,0.396552,0.063562,1825,23.701299,3952.0,1109.0,1.0
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...","[human, minds, a, fascination, in, itself, car...",72.0,37.0,0.321101,0.075433,1445,18.062500,2680.0,710.0,7.0
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...,"[introduction, ai, is, rapidly, evolving, in, ...",74.0,36.0,0.345455,0.064140,1715,20.176471,3641.0,987.0,3.0
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,"[anything, that, could, give, rise, to, smarte...",86.0,28.0,0.508772,0.068841,1656,17.431579,3181.0,882.0,18.0
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,"[machine, intelligence, is, the, last, inventi...",71.0,29.0,0.420000,0.055928,1788,22.632911,3433.0,925.0,16.0


Average Word Length


In [ ]:
df['Average Word Length'] = np.NaN

def word_count(word):
    count = 0
    for w in word:
      if w !=' ':
        count += 1
    return count 

for i in range (0,len(df)):
  df['Average Word Length'][i] = word_count(' '.join(df['Transformed_text'][i])) / df['word_counts'][i]


In [34]:
df.head(5)

,URL_ID,URL,Title,Text,Transformed_text,Positive Score,Negative Score,Polarity Score,Subjectivity Score,word_counts,average number of words per sentence,Syllable Count,Complex Word Count,Personal Pronouns,Average Word Length
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...,"[introduction, if, anything, kills, over, 10, ...",81.0,35.0,0.396552,0.063562,1825,23.701299,3952.0,1109.0,1.0,5.556164
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...","[human, minds, a, fascination, in, itself, car...",72.0,37.0,0.321101,0.075433,1445,18.062500,2680.0,710.0,7.0,4.728720
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...,"[introduction, ai, is, rapidly, evolving, in, ...",74.0,36.0,0.345455,0.064140,1715,20.176471,3641.0,987.0,3.0,5.365598
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,"[anything, that, could, give, rise, to, smarte...",86.0,28.0,0.508772,0.068841,1656,17.431579,3181.0,882.0,18.0,4.783213
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,"[machine, intelligence, is, the, last, inventi...",71.0,29.0,0.420000,0.055928,1788,22.632911,3433.0,925.0,16.0,4.964206


Analysis of Readability
Average Sentence Length = the number of words / the number of sentences
Percentage of Complex words = the number of complex words / the number of words 
Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)


In [ ]:
df['Average Sentence Length'] = np.nan
df['Percentage of Complex Words'] = np.nan
df['Fog Index'] = np.nan   

for i in range(0,len(df)):
    
  df['Average Sentence Length'][i]  =   df['word_counts'][i] / len(nltk.sent_tokenize(df['Text'][i]))
  df['Percentage of Complex Words'][i] = df['Complex Word Count'][i] / df['word_counts'][i] 
  df['Fog Index'][i] = 0.4 * (df['Average Sentence Length'][i] + df['Percentage of Complex Words'][i])

In [36]:
df.head(5)

,URL_ID,URL,Title,Text,Transformed_text,Positive Score,Negative Score,Polarity Score,Subjectivity Score,word_counts,average number of words per sentence,Syllable Count,Complex Word Count,Personal Pronouns,Average Word Length,Average Sentence Length,Percentage of Complex Words,Fog Index
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n\n“If anything kills over 10 mil...,"[introduction, if, anything, kills, over, 10, ...",81.0,35.0,0.396552,0.063562,1825,23.701299,3952.0,1109.0,1.0,5.556164,23.701299,0.607671,9.723588
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...","[human, minds, a, fascination, in, itself, car...",72.0,37.0,0.321101,0.075433,1445,18.062500,2680.0,710.0,7.0,4.728720,18.062500,0.491349,7.421540
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\n\nAI is rapidly evolving in the ...,"[introduction, ai, is, rapidly, evolving, in, ...",74.0,36.0,0.345455,0.064140,1715,20.176471,3641.0,987.0,3.0,5.365598,20.176471,0.575510,8.300792
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,"[anything, that, could, give, rise, to, smarte...",86.0,28.0,0.508772,0.068841,1656,17.431579,3181.0,882.0,18.0,4.783213,17.431579,0.532609,7.185675
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,"[machine, intelligence, is, the, last, inventi...",71.0,29.0,0.420000,0.055928,1788,22.632911,3433.0,925.0,16.0,4.964206,22.632911,0.517338,9.260100


Rename columns as in outpit file

In [37]:
df.rename(columns = {'Positive Score':'POSITIVE SCORE','Negative Score':'NEGATIVE SCORE' , 'Polarity Score' : 'POLARITY SCORE', 'Subjectivity Score' : 'SUBJECTIVITY SCORE', 'Average Sentence Length' : 'AVG SENTENCE LENGTH', 'Percentage of Complex Words' : 'PERCENTAGE OF COMPLEX WORDS', 'Fog Index' : 'FOG INDEX','average number of words per sentence' : 'AVG NUMBER OF WORDS PER SENTENCE', 'Complex Word Count' : 'COMPLEX WORD COUNT', 'word_counts' : 'WORD COUNT', 'Syllable Count' : 'SYLLABLE PER WORD' , 'Personal Pronouns' : 'PERSONAL PRONOUNS' ,'Average Word Length' : 'AVG WORD LENGTH'}, inplace = True)

Drop column Title and Text

In [38]:
df = df.drop({'Title','Text','Transformed_text'}, axis = 1)

In [39]:
df.head(5)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,WORD COUNT,AVG NUMBER OF WORDS PER SENTENCE,SYLLABLE PER WORD,COMPLEX WORD COUNT,PERSONAL PRONOUNS,AVG WORD LENGTH,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX
0,37,https://insights.blackcoffer.com/ai-in-healthc...,81.0,35.0,0.396552,0.063562,1825,23.701299,3952.0,1109.0,1.0,5.556164,23.701299,0.607671,9.723588
1,38,https://insights.blackcoffer.com/what-if-the-c...,72.0,37.0,0.321101,0.075433,1445,18.062500,2680.0,710.0,7.0,4.728720,18.062500,0.491349,7.421540
2,39,https://insights.blackcoffer.com/what-jobs-wil...,74.0,36.0,0.345455,0.064140,1715,20.176471,3641.0,987.0,3.0,5.365598,20.176471,0.575510,8.300792
3,40,https://insights.blackcoffer.com/will-machine-...,86.0,28.0,0.508772,0.068841,1656,17.431579,3181.0,882.0,18.0,4.783213,17.431579,0.532609,7.185675
4,41,https://insights.blackcoffer.com/will-ai-repla...,71.0,29.0,0.420000,0.055928,1788,22.632911,3433.0,925.0,16.0,4.964206,22.632911,0.517338,9.260100


Write output to "Output Data Structure.xlsx" file

In [40]:
df.to_excel("Output Data Structure.xlsx", index=False)